In [1]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers
import pandas as pd
import tarfile
import os
import gc
import random

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D,Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, Callback, ReduceLROnPlateau

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',5000)
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_31/94654923.py:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [25]:
# Charger les données
data = pd.read_csv("/kaggle/input/balanced2/reviews_balanced1.csv", encoding='utf-8')

# Supprimer la colonne 'Rating'
data = data.drop(['Rating'], axis=1)

# Mélanger les lignes de la DataFrame
# data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data.head(5)

,Review
0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!"
1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque."
2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR."
3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème."
4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !"


# Precprocessing

In [26]:
import re
import string
import nltk
from nltk.corpus import stopwords

def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()

    # Supprimer les caractères spéciaux et les emojis
#     text = emoji.demojize(text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r':[a-z_]+:', ' ', text)

    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenisation
    tokens = text.split()

    # Supprimer les stopwords
    stop_words = set(stopwords.words('french'))
    tokens = [token for token in tokens if token not in stop_words]

#     # Lemmatisation avec spaCy
#     doc = nlp(text)
#     tokens = [token.lemma_ for token in doc]

#     # Stemming
#     stemmer = SnowballStemmer("french")
#     tokens = [stemmer.stem(word) for word in tokens]

    return ' '.join(tokens)

In [27]:
data['Review_cleaned'] = data['Review'].apply(preprocess_text)

# Hugging face models

In [5]:
pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 15.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=19c1e77b7550577d80a632f10173e516ef07f212c11f519a519c7720f0f6c062
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Note: you may need to restart the kernel to use updated packages.


In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import TFAutoModel, AutoTokenizer
import transformers as trf

tokenizer = AutoTokenizer.from_pretrained("DohaData/gpt2-base-french-finetuned")

model = TFAutoModel.from_pretrained("DohaData/gpt2-base-french-finetuned",from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'score.weight', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the

In [41]:
def gpt2_embedding(txt):
    idx = tokenizer.encode(txt, max_length=1024, truncation=True) # creating tokens
    if len(idx) >= 1024:
        return None
    idx = np.array(idx)[None,:] # converting to 2d array

    emb = model(idx) # bert layer
    hidden = np.array(emb[0][0]) # batch output of last_hidden_state

    sent_emb = hidden.mean(0) # creating mean vector
    return sent_emb

# W2V

In [40]:
import numpy as np
import gensim

# Load pre-trained Word2Vec model
model_w2v = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/french-w2v/frWac_no_postag_phrase_500_cbow_cut100.bin', binary=True)

def bert_embedding(txt):
    # Split text into individual words
    words = txt.split()

    # Initialize empty array to store embeddings
    embeddings = np.zeros((len(words), model_w2v.vector_size))

    # Loop through each word and get its embedding
    for i, word in enumerate(words):
        if word in model_w2v:
            embeddings[i] = model_w2v[word]

    # Compute the mean embedding across all words in the text
    sent_emb = np.mean(embeddings, axis=0)

    return sent_emb

In [42]:
from tqdm.autonotebook import tqdm

sent_matrix = np.array([gpt2_embedding(text) for text in tqdm(data['Review_cleaned'][0:300])] , dtype=object)
sent_matrix.shape

  0%|          | 0/300 [00:00<?, ?it/s]

(300, 768)

# Glossaire

In [43]:
RESIL = [
        "Quitté pour un autre opérateur avec une portabilité le 11 janvier.",
         "déjà j'avais quitté red mobile",
         "J’ai essayé résilier",
         "Je me suis retrouvée avec mon contrat résilié.",
          "Lorsque je demande une résiliation.",
         "J'ai demandé la résiliation de ma box",
         "Le client exprime son souhait de résilier son offre et mentionne que l'opérateur demande des frais de résiliation.",
         "Alors pour la résiliation bordel .",
         "J'ai demandé la résiliation de ma box Internet le 14 juin.",
        "lorsque vous résiliez votre contrat, n'applique pas la résiliation",
        "La résiliation sera automatique de ce fait.",
        " Demandes de résiliation par formulaire",
        " j'ai voulu resilier.",
        "RESILIATION.",
        " apres resilation de red by sfr"
        

        ]

image_de_marque = [

" Des voleurs et des menteurs. Impossibilité de les joindre sauf par messenger. Attention arnaque",
"red SFR box à fuir.",
"Je pars donc de chez eux car j'en peux plus, c'est le coup de grâce !! Se sont des voleurs !!! A fuir !!!",
"Quitté pour un autre opérateur",
"Fuyez ou changez.",
"A fuir, je ne recommande pas, on se sent volé!",
"Je pense changer d’opérateur.",
"Fuyez ou changez."

                  ]

service_client = [ 
    
"Impossible de joindre un conseillé N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux."
,"Le client a contacté plusieurs services clients, mais n'a obtenu aucune réponse satisfaisante. ."
,"Contact par chat uniquement avec des conseillers incompétents qui ne savent pas pourquoi la demande est rejetée, et qui vous renvoient au site ou à faire un courrier à SFR. Le service client",
"Service client deplorable.. Elle a fini par mettre fin à la conversation sans attendre réponse de ma part.",
"ont peut que communiquer avec eux que par message SMS",
"Lorsque je les ai contacté",
"J'appelle ou je tchat quotidiennement avec le service clientèle qui n'est toujours pas en mesure de me donner un rdv pour installer et raccorder ma ligne.",
"Impossible de parler à un humain de vive voix ou de face à face.",
"Et impossible après 29 tentatives de les joindre pour remboursement de la période du 11 au 30. Drôle de manières !!!",
"c'est inacceptable", "je choisis les offres qui sont respectées a vie", "et red ne fait plus partie de ces revendeurs.",
"j'ai fait 6 demandes et de multiples appels et chat en ligne et la demande n'aboutit pas. Le SAV est totalement déficient !!!",
"Service client incompétent ",
"un coup de fil, et généralement le problème est résolu rapidement",
"Service client d’une incompétence… à ce niveau là c’est du jamais vu. Une fois sur 2 on vous raccroche au nez.",
"Service client incompetent",
"Au bout de plus d’un mois à joindre leurs services quotidiennement.",
"Le service consommateur lit un script et ne sait pas quoi dire si on sort du script.",
"Aucun vrai conseiller, seulement un Bot. ça fait deux heures que j'essaie de résoudre un problème, et toutes les deux minutes ça se réinitialise et me repose les mêmes questions.",
"SAV compétent et professionnel",
"des dizaines d'échanges avec le service client, sur l'application et par téléphone",
"Le service client inactif ",


]



forfait_offre = [ 

"On vous promet une offre à vie et hop d un couo de baguette magique on vous augmente de 30%",
"Pub TV et sans prévenir",
"Je vous propose de profitez de 15€ offerts lors de votre souscription d'abonnement chez Red. (15€ pour Red Box et 10€ pour Red mobile)",
"quand on sait que a la poste le forfait est a 9.99 euro pour 50 go dont 15 go internationale et A VIE.",
 "Et lorsque vous avez quelqu'un, il vous promet un geste commercial que vous n'avez jamais, simplement pour se débarrasser de vous!",
" Netflix mais au final ils ne m'offrent ni Netflix ni le bouquet, super de faire perdre du temps aux gens pour rien.",
"Pour mes deux offres, je me retrouve régulièrement avec des options pour lesquelles je n'ai fais aucunes demandes",
"Il vous change votre abonnement sans vous prévenir pour une meilleure offre sois disant avec une prime en plus de 3 €",
"Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien."
" j'ai un forfait mobile soit chez SFR et par la suite red",
]

facturation = [

"mon problème de facturation a été géré"
"Je n'ai jamais rencontré de problèmes avec la facturation ",
"Me facture plus cher",
"Les factures reçues sont supérieures à ce qui a été convenu",
"Après plusieurs minutes à essayer d'avoir un details de facture la facture est correcte.",
"Le nombre de fois que je me suis retrouvée avec des factures augmentées on ne sait pas pourquoi... Ils vous rajoutent des options sans que vous ayez rien demandé !",
"Ils m'ont changé mon forfait sans aucune demande de ma part (toujours pareil sans aucun consentement de ma part), bien évidemment beaucoup plus cher !!!!",
"Quitté pour un autre opérateur avec une portabilité le 11 janvier, ils retirent sur ton compte l'intégralité forfait de janvier et des portions telles que Deezer.",
"Leur prix augmentent sans notification de leur part de quelques euros chaque année.",
"tous les mois paye et non remboursé pour une box dans un carton.",
"surfacturation mensuelle d'une option tv que je n ai pas",
"Prix en perpetuelle augmentation",
"Elle n'a pas été effective et j'ai été facturée les mois de juin, juillet et août. À ce jour, donc en octobre, j'attends toujours le remboursement après des dizaines d'échanges avec le service client, sur l'application et par téléphone, qui me confirment à chaque fois que je vais être remboursée mais ce est absolument pas le cas. attends toujours.",
"J'ai été facturée les mois de juin, juillet et août.",
"Je me retrouve donc certains mois avec +3€ par-ci, +5€ par là.",
"Je suis très satisfait de la façon dont mon fournisseur de téléphonie traite les problèmes de facturation.",
"Je suis très mécontent de la facturation exorbitante",



]

debit_internet = [
    
"La vitesse de l'internet est vraiment rapide et stable!"
"Problème sur ma fibre optique",
"pas moyen de passer un coup de fil sans coupure, ils m'ont dit configurer en appels wifi pour éviter le probleme, ok mais dans ce cas à quoi cela sert il que je paye un abonnement chez eux. minable",
" sans internet alors que c'est le seul outil nécessaire.",
"ces 4g box sont les plus éclatées que j'ai jamais eu de ma vie",
"même sans ces bugs la connexion n'est pas stable",
"Problème sur ma ligne fixe FIBRE apparu le 1 octobre",
"Problème sur ma fibre optique",
"la fibre plante un jour sur deux.",
"une offre fibre",
"pres 6 mois de fibre chez Red by Sfr",
"Problèmes de débit"
]

installation_service_technique = [
    
                
" un technicien est programmé pour venir le lendemain 09/03 entre 10h et 12h. Évidemment personne ne vient....",
"un rdv pour installer et raccorder ma ligne.",
 "2 semaines d'attentes pour avoir un technicien.",
" aucune trace du technicien le jour J.",
" technicien absent, problème toujours présent.",
"Et les techniciens sont tous en sous-traitances ce qui fait qu'ils ne sont pas investis.",
 "Le technicien ne vient pas au rdv.",
 "Des rendez-vous non honorés de la part des techniciens, un problème qui vient toujours d’ailleurs.",
"Problèmes techniques récurrents avec la fibre. Ça fait 3 mois que nous l'utilisons, nous en sommes à notre 5ème dépannage pour coupure l'Internet (dont 2 la même semaine).",
"30 minutes d'attente technicien qui ne se présente pas et pas de disponibilité pour un dépannage de box qui coupe sans arrêt",
"technicien qui ne se présente pas",
"il faut appeler un technicien, qui la plupart du temps met plus d'une semaine à arriver et ne sert à rien",
                        
                
    ]


contrat = [
"Le contrat téléphonique est facile à gérer"
" changement de titulaire sur une ligne existante, ."   ,
"Tiens tiens tiens ... 2 mois.... c'est justement le délai légal dans lequel le FAI doit impérativement avoir effectué le raccordement. Au delà il est susceptible d'avoir des problèmes juridiques et comme par hasard on me propose ce jour là d'annuler et recommencer la procédure ...",
 "ont tendance a vouloir imposer des options",
 "On fait des erreurs sur votre contrat.",
"la fidélité ne paye pas même pour un service sans engagement.",
"lors de ma souscription j'étais sous liste rouge,",
" sans pour autant changer le contrat.",
"rien de cela n'est stipulé dans mon contrat",
"J'ai demandé la résiliation de mon contrat.",
"On fait des erreurs sur votre contrat… incroyable!"


]
    

couverture_reseau = ["de la couverture réseau de l'opérateur.",
                     "Le réseau est très lent dans ma région et j'ai souvent des problèmes de connexion.",
                     "La qualité du réseau est très variable selon l'endroit où je me trouve.",
                     "Le service réseau est très cher par rapport à la qualité de la connexion.",
                     "J'ai rencontré des problèmes pour passer des appels et envoyer des SMS dans ma région.",
                     "Le réseau de l'opérateur est très fiable et n'a jamais été interrompu.",
                     "Le débit Internet est très instable et change souvent.",
                     "Je suis souvent coupé lors de mes appels téléphoniques en raison d'un signal faible.",
                     "Le réseau est souvent en panne et le service technique ne semble pas capable de le réparer.",
                     "La couverture réseau de l'opérateur est très limitée par rapport à la concurrence."
                     " le réseau SFR nul ",
                     "Problèmes de coupures intempestives du réseau",
                    
                    ]

# Preproc and Word Embedding

In [44]:
# Image de marque
IMAGE_MARQUE = [preprocess_text(text) for text in image_de_marque]

# Service client
SERVICE_CLI = [preprocess_text(text) for text in service_client]

# Forfait/Offre
FORFAIT_OFFRE = [preprocess_text(text) for text in forfait_offre]

# Facturation
FACTURATION = [preprocess_text(text) for text in facturation]

# Débit/Internet
DEBIT_INTERNET = [preprocess_text(text) for text in debit_internet]

# Installation/Service technique
INSTALLATION_SERVICE_TECHNIQUE = [preprocess_text(text) for text in installation_service_technique]

# Contrat
CONTRAT = [preprocess_text(text) for text in contrat]

# Couverture/Réseau
COUVERTURE_RESEAU = [preprocess_text(text) for text in couverture_reseau]

In [45]:
import numpy as np

# Image de marque
IMAGE_MARQUE = np.array([gpt2_embedding(text) for text in image_de_marque])

# Service client
SERVICE_CLI = np.array([gpt2_embedding(text) for text in service_client])

# Forfait/Offre
FORFAIT_OFFRE = np.array([gpt2_embedding(text) for text in forfait_offre])

# Facturation
FACTURATION = np.array([gpt2_embedding(text) for text in facturation])

# Débit/Internet
DEBIT_INTERNET = np.array([gpt2_embedding(text) for text in debit_internet])

# Installation/Service technique
INSTALLATION_SERVICE_TECHNIQUE = np.array([gpt2_embedding(text) for text in installation_service_technique])

# Contrat
CONTRAT = np.array([gpt2_embedding(text) for text in contrat])

# Couverture/Réseau
COUVERTURE_RESEAU = np.array([gpt2_embedding(text) for text in couverture_reseau])

In [46]:
COUVERTURE_RESEAU

array([[ 1.2896296 , -0.8483925 ,  0.49118403, ..., -0.52803785,
        -0.10710566,  0.8912769 ],
       [ 0.44460964, -1.3341212 , -0.48331934, ..., -1.9188315 ,
        -0.14350066,  1.2109185 ],
       [-0.96131897, -0.47234106, -1.430458  , ..., -1.8049917 ,
        -0.5735003 ,  2.5874786 ],
       ...,
       [ 0.43832973, -0.93038434, -1.0339149 , ..., -1.8719771 ,
        -0.03326508,  0.12061826],
       [ 1.9109437 ,  0.1437227 , -1.0256742 , ..., -1.701226  ,
        -1.2583224 ,  0.7488008 ],
       [ 0.5216077 , -3.301291  ,  1.4131327 , ..., -0.9224047 ,
        -0.44460642, -0.26297575]], dtype=float32)

In [133]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcul de la similarité cosinus pour la catégorie Image de marque
IMAGE_MARQUE_EMB = [t for t in IMAGE_MARQUE]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(IMAGE_MARQUE_EMB[1][None,:], IMAGE_MARQUE_EMB[4][None,:])))

# Calcul de la similarité cosinus pour la catégorie Service client
SERVICE_CLI_EMB = [t for t in SERVICE_CLI]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(SERVICE_CLI_EMB[1][None,:], SERVICE_CLI_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Forfait/Offre
FORFAIT_OFFRE_EMB = [t for t in FORFAIT_OFFRE]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(FORFAIT_OFFRE_EMB[1][None,:], FORFAIT_OFFRE_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Facturation
FACTURATION_EMB = [t for t in FACTURATION]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(FACTURATION_EMB[1][None,:], FACTURATION_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Débit/Internet
DEBIT_INTERNET_EMB = [t for t in DEBIT_INTERNET]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(DEBIT_INTERNET_EMB[1][None,:], DEBIT_INTERNET_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Installation/Service technique
INSTALLATION_SERVICE_TECHNIQUE_EMB = [t for t in INSTALLATION_SERVICE_TECHNIQUE]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(INSTALLATION_SERVICE_TECHNIQUE_EMB[1][None,:], INSTALLATION_SERVICE_TECHNIQUE_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Contrat
CONTRAT_EMB = [t for t in CONTRAT]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(CONTRAT_EMB[1][None,:], CONTRAT_EMB[2][None,:])))

# Calcul de la similarité cosinus pour la catégorie Couverture/Réseau
COUVERTURE_RESEAU_EMB = [t for t in COUVERTURE_RESEAU]
print('Cosine similarity of given samples: {}'.format(cosine_similarity(COUVERTURE_RESEAU_EMB[1][None,:], COUVERTURE_RESEAU_EMB[2][None,:])))

Cosine similarity of given samples: [[0.2850278]]
Cosine similarity of given samples: [[0.6006048]]
Cosine similarity of given samples: [[0.36958888]]
Cosine similarity of given samples: [[0.23049128]]
Cosine similarity of given samples: [[0.43884817]]
Cosine similarity of given samples: [[0.41074163]]
Cosine similarity of given samples: [[0.22110642]]
Cosine similarity of given samples: [[0.63482225]]


# Cosine Similarity

In [134]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Calcul des scores de similarité cosinus pour chaque catégorie
IMAGE_MARQUE_MEAN = IMAGE_MARQUE.mean(0)[None,:]
IMAGE_MARQUE_SCORES = cosine_similarity(sent_matrix, IMAGE_MARQUE_MEAN)

SERVICE_CLI_MEAN = SERVICE_CLI.mean(0)[None,:]
SERVICE_CLI_SCORES = cosine_similarity(sent_matrix, SERVICE_CLI_MEAN)

FORFAIT_OFFRE_MEAN = FORFAIT_OFFRE.mean(0)[None,:]
FORFAIT_OFFRE_SCORES = cosine_similarity(sent_matrix, FORFAIT_OFFRE_MEAN)

FACTURATION_MEAN = FACTURATION.mean(0)[None,:]
FACTURATION_SCORES = cosine_similarity(sent_matrix, FACTURATION_MEAN)

DEBIT_INTERNET_MEAN = DEBIT_INTERNET.mean(0)[None,:]
DEBIT_INTERNET_SCORES = cosine_similarity(sent_matrix, DEBIT_INTERNET_MEAN)

INSTALLATION_SERVICE_TECHNIQUE_MEAN = INSTALLATION_SERVICE_TECHNIQUE.mean(0)[None,:]
INSTALLATION_SERVICE_TECHNIQUE_SCORES = cosine_similarity(sent_matrix, INSTALLATION_SERVICE_TECHNIQUE_MEAN)

CONTRAT_MEAN = CONTRAT.mean(0)[None,:]
CONTRAT_SCORES = cosine_similarity(sent_matrix, CONTRAT_MEAN)

COUVERTURE_RESEAU_MEAN = COUVERTURE_RESEAU.mean(0)[None,:]
COUVERTURE_RESEAU_SCORES = cosine_similarity(sent_matrix, COUVERTURE_RESEAU_MEAN)

# Création du DataFrame pour stocker les scores de similarité
cosine_score = pd.DataFrame()
cosine_score['id'] = range(len(sent_matrix))
cosine_score['Image de marque'] = cosine_similarity(sent_matrix, IMAGE_MARQUE_MEAN)
cosine_score['Service client'] = cosine_similarity(sent_matrix, SERVICE_CLI_MEAN)
cosine_score['Forfait/Offre'] = cosine_similarity(sent_matrix, FORFAIT_OFFRE_MEAN)
cosine_score['Facturation'] = cosine_similarity(sent_matrix, FACTURATION_MEAN)
cosine_score['Débit/Internet'] = cosine_similarity(sent_matrix, DEBIT_INTERNET_MEAN)
cosine_score['Installation/Service technique'] = cosine_similarity(sent_matrix, INSTALLATION_SERVICE_TECHNIQUE_MEAN)
cosine_score['Contrat'] = cosine_similarity(sent_matrix, CONTRAT_MEAN)
cosine_score['Couverture/Réseau'] = cosine_similarity(sent_matrix, COUVERTURE_RESEAU_MEAN)

# Trouver les trois catégories les plus similaires pour chaque phrase
top3_categories = cosine_score[['Image de marque', 'Service client', 'Forfait/Offre', 'Facturation', 'Débit/Internet', 'Installation/Service technique', 'Contrat', 'Couverture/Réseau']].apply(lambda x: x.nlargest(3).index.tolist(), axis=1)

# Ajouter les colonnes au DataFrame
cosine_score['Category'] = top3_categories.apply(lambda x: x[0])
cosine_score['Category2'] = top3_categories.apply(lambda x: x[1] if len(x)>1 else None)
cosine_score['Category3'] = top3_categories.apply(lambda x: x[2] if len(x)>2 else None)

cosine_score.head(22)

,id,Image de marque,Service client,Forfait/Offre,Facturation,Débit/Internet,Installation/Service technique,Contrat,Couverture/Réseau,Category,Category2,Category3
0,0,0.297422,0.132818,0.196952,0.169167,0.174460,0.175078,0.087510,0.186455,Image de marque,Forfait/Offre,Couverture/Réseau
1,1,0.478226,0.423053,0.420811,0.377039,0.307905,0.296636,0.234563,0.241084,Image de marque,Service client,Forfait/Offre
2,2,0.123820,0.227855,0.333281,0.419726,0.159280,0.157039,0.248357,0.136543,Facturation,Forfait/Offre,Contrat
3,3,0.493108,0.458135,0.418212,0.390650,0.185043,0.265909,0.296437,0.126379,Image de marque,Service client,Forfait/Offre
4,4,0.354563,0.372069,0.417688,0.365755,0.174508,0.253534,0.276425,0.117717,Forfait/Offre,Service client,Facturation
5,5,0.390638,0.418595,0.471131,0.486260,0.289788,0.302069,0.345692,0.223344,Facturation,Forfait/Offre,Service client
6,6,0.524444,0.409167,0.406778,0.360555,0.194691,0.292103,0.315597,0.142681,Image de marque,Service client,Forfait/Offre
7,7,0.406440,0.352030,0.355583,0.343570,0.190052,0.370128,0.284868,0.152109,Image de marque,Installation/Service technique,Forfait/Offre
8,8,0.407106,0.483342,0.488693,0.519843,0.332488,0.280335,0.332232,0.305145,Facturation,Forfait/Offre,Service client
9,9,0.462568,0.515702,0.452009,0.476281,0.253877,0.341998,0.383509,0.224237,Service client,Facturation,Image de marque


In [135]:
import gc

# cosine_score['top_categories'] = cosine_score[['Facturation/resiliance','Service Client','Technique/Installation',"Changement d'offre"]].idxmax(axis=1) #finding the column which has maximum value and retunrning the column name (this becomes the label for the text)
df = cosine_score

del cosine_score #deleting as we dont need this df anymore
gc.collect()

df.head()

,id,Image de marque,Service client,Forfait/Offre,Facturation,Débit/Internet,Installation/Service technique,Contrat,Couverture/Réseau,Category,Category2,Category3
0,0,0.297422,0.132818,0.196952,0.169167,0.174460,0.175078,0.087510,0.186455,Image de marque,Forfait/Offre,Couverture/Réseau
1,1,0.478226,0.423053,0.420811,0.377039,0.307905,0.296636,0.234563,0.241084,Image de marque,Service client,Forfait/Offre
2,2,0.123820,0.227855,0.333281,0.419726,0.159280,0.157039,0.248357,0.136543,Facturation,Forfait/Offre,Contrat
3,3,0.493108,0.458135,0.418212,0.390650,0.185043,0.265909,0.296437,0.126379,Image de marque,Service client,Forfait/Offre
4,4,0.354563,0.372069,0.417688,0.365755,0.174508,0.253534,0.276425,0.117717,Forfait/Offre,Service client,Facturation


In [136]:
df['Review'] = data['Review'][0:300] #Earlier we have defined num_sentences
df.drop(['Image de marque', 'Service client', 'Forfait/Offre', 'Facturation', 'Débit/Internet', 'Installation/Service technique', 'Contrat', 'Couverture/Réseau'],axis=1,inplace =True) 
df1 = df[['id','Review','Category','Category2','Category3']]
df1.head()

,id,Review,Category,Category2,Category3
0,0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!",Image de marque,Forfait/Offre,Couverture/Réseau
1,1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque.",Image de marque,Service client,Forfait/Offre
2,2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR.",Facturation,Forfait/Offre,Contrat
3,3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.",Image de marque,Service client,Forfait/Offre
4,4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !",Forfait/Offre,Service client,Facturation


In [120]:
df1['top_categories'] = df1['top_categories'].apply(lambda x: ','.join(map(str, x))).str.replace(',', ';')

In [65]:
df1['Review'] = df1['Review'].str.replace('\n', ' ')
df1['Review'] = df1['Review'].str.replace(',', ' ')

In [137]:
df1.to_csv("GPT2_COSINE.csv", encoding='utf-8-sig',header=1,sep=';')

In [127]:
df1.head(5)

,id,Review,top_categories
0,0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!",Image de marque;Forfait/Offre;Couverture/Réseau
1,1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque.",Image de marque;Service client;Forfait/Offre
2,2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR.",Facturation;Forfait/Offre;Contrat
3,3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.",Image de marque;Service client;Forfait/Offre
4,4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !",Forfait/Offre;Service client;Facturation


In [ ]:
warnings.filterwarnings('ignore')

lenx = []
for txt in df1['Review']:
  encodes = tokenizer.encode_plus(txt) #we have already defined the tokenizer
  lenx.append(len(encodes['input_ids']))


#ploting the length distribution
plt.figure(figsize=(20,7))
sns.distplot(lenx)
plt.xlabel('Token counts')
plt.show()

In [ ]:
print('Max len: {}'.format(np.max(lenx)))
MAX_LEN = np.max(lenx)+10

In [ ]:
df1['Facturation/resiliance'] = [1 if 'Facturation/resiliance' in i else 0 for i in df1['top_categories']]
df1['Service Client'] = [1 if 'Service Client' in i else 0 for i in df1['top_categories']]
df1['Technique/Installation'] = [1 if 'Technique/Installation' in i else 0 for i in df1['top_categories']]
# df1["Changement d'offre"] = [1 if "Changement d'offre" in i else 0 for i in df1["top_categories"]]
df1['Debit Internet'] = [1 if 'Debit Internet' in i else 0 for i in df1['top_categories']]
df1.head()

In [ ]:
df1.top_categories.value_counts()

In [ ]:
def encode(texts, MAX_LEN):
  input_ids =[]
  attention_masks =[]
  token_type_ids =[]

  for text in tqdm(texts):
    tokens = tokenizer.encode_plus(text, max_length= MAX_LEN, padding='max_length', return_tensors='tf',truncation=True)
    input_ids.append(tokens['input_ids'][0])
    attention_masks.append(tokens['attention_mask'][0])
    token_type_ids.append(tokens['token_type_ids'][0])
    # print(tokens['input_ids'][0])


  return np.array(input_ids), np.array(attention_masks), np.array(token_type_ids)

In [ ]:
input_ids_train, attention_mask_train, token_type_ids_train =encode(df1['Review'],MAX_LEN)
input_ids_train.shape

In [ ]:
y_fact = np.array(df1['Facturation/resiliance'])
y_client = np.array(df1['Service Client'])
y_tech = np.array(df1['Technique/Installation'])
# y_offre = np.array(df1["Changement d'offre"])
y_internet = np.array(df1['Debit Internet'])

In [ ]:
#Hyperparameters
batch_size=4
epochs=10
lr=0.001
SEED = 5520

In [ ]:
#Model Building
def build_model():

    input_id = Input(shape =(MAX_LEN,),dtype =tf.int32)
    att_mask = Input(shape =(MAX_LEN,),dtype =tf.int32)
    token_type = Input(shape =(MAX_LEN,),dtype =tf.int32)

    embed = bert([input_id,att_mask,token_type])

    cls_output = embed[0][:,0,:] #embedding of the [CLS] token


    facture_out = Dropout(0.1)(cls_output)
    facture_out = Dense(1, activation= 'sigmoid')(facture_out)

    client_out = Dropout(0.1)(cls_output)
    client_out = Dense(1, activation= 'sigmoid')(client_out)

    tech_out = Dropout(0.1)(cls_output)
    tech_out = Dense(1, activation= 'sigmoid')(tech_out)

    offre_out = Dropout(0.1)(cls_output)
    offre_out = Dense(1, activation= 'sigmoid')(offre_out)

    internet_out = Dropout(0.1)(cls_output)
    internet_out = Dense(1, activation= 'sigmoid')(internet_out)


    model = Model(inputs=[input_id,att_mask,token_type], outputs = [facture_out,client_out,tech_out,offre_out,internet_out])
    optimizer = Adam(learning_rate=lr)

    model.compile(loss = 'binary_crossentropy',optimizer = optimizer,metrics=['accuracy'])

    return model

In [ ]:
#model building
bert_model = build_model()

#model training

bert_model.fit([input_ids_train,attention_mask_train,token_type_ids_train],[y_fact,y_client,y_tech,y_offre,y_internet],
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2)